1. Implementar o algoritmo de Similaridade do Cosseno;

In [2]:
import numpy as np
import math
def cosinesim(A,B):
    sum = 0
    modA = 0
    modB = 0
    for i in range(len(A)):
        sum = sum + A[i]*B[i]
        modA = modA + (A[i])**2
        modB = modB + (B[i])**2
    cosine = sum/(math.sqrt(modA)*math.sqrt(modB))
    return cosine


2. Implementar um Sistema de Recomendações, usando a técnica de Filtros Colaborativos baseados no usuário;

In [3]:
#Preparação do dataframe
import pandas as pd
df = pd.read_csv('/home/hub/Project/resident-profiles/Erick_E/Modulo_02/Jupyter/ml-100k/u.data', sep='\t', names=['Usuário','Filme','Avaliação', 'Data'])
generos = pd.read_csv('/home/hub/Project/resident-profiles/Erick_E/Modulo_02/Jupyter/ml-100k/u.item', sep='|',encoding='latin-1', names=['Nome','Data','','IMDB','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19'])
merge=df.set_index(['Filme']).astype('int').join(generos,how = 'left', lsuffix='left', rsuffix='right')
merge.index.name = 'Filme'
usersgen = merge.set_index(['Usuário'])

In [4]:
for i in range(19):
    usersgen[str(i+1)]=usersgen[str(i+1)]*usersgen['Avaliação'] #multiplica a nota dada pela avaliação pelo vetor de gêneros de cada filme
usersgen = usersgen.replace(0, np.NaN) #Os zeros são substituídos por Nan para a média apenas calcular os filmes que possuem valor de gênero 1
nuser = len(usersgen.groupby(['Usuário']).mean()) #usersgen.groupby(['Usuário']).mean(): Calcula a média de nota para cada gênero de cada usuário
avgen = np.zeros((nuser,19)) #inicialização do vetor que possui uma linha para cada usuário e uma coluna para cada gênero de filme
for i in range(nuser):
    avgen[i]=list(usersgen.groupby(['Usuário']).mean().loc[i+1][3:22].replace(np.NaN,0))#lista com os valores médios por gênero para cada usuário, os Nan são resubstituidos por zero

/tmp/ipykernel_5267/66140920.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  nuser = len(usersgen.groupby(['Usuário']).mean()) #usersgen.groupby(['Usuário']).mean(): Calcula a média de nota para cada gênero de cada usuário
/tmp/ipykernel_5267/66140920.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  avgen[i]=list(usersgen.groupby(['Usuário']).mean().loc[i+1][3:22].replace(np.NaN,0))#lista com os valores médios por gênero para cada usuário, os Nan são resubstituidos por zero
/tmp/ipykernel_5267/66140920.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a

In [5]:
user = input("Insira um usuário:")
similaridade = np.zeros(nuser)
for i in range(nuser):
    similaridade[i] = cosinesim(avgen[int(user)-1],avgen[i-1]) #calcula a similaridade entre o usuario e todos os outros
mostsimilaruser = pd.DataFrame(similaridade, columns = ['Similaridade']).sort_values(by = ['Similaridade']).index[1] 
#O usuario mais similar é o que possui o maior valor de similaridade calculado pelo cosseno. Deve-se pular o primeiro valor pois o mais similar é o próprio usuário.
print("Usuário mais semelhante:",mostsimilaruser)
filmesuser = list(df.set_index(['Usuário']).loc[int(user)]['Filme']) #filmes assistidos pelo usuario
filmesmostsimilaruser = list(df.set_index(['Usuário']).loc[mostsimilaruser]['Filme']) #filmes assistidos pelo usuario mais similar
filmesiguais = list(set(filmesuser) & set(filmesmostsimilaruser)) #filmes assistidos pelos 2
filmesindic = df.set_index(['Usuário']).loc[mostsimilaruser].sort_values(by =['Avaliação'],ascending = False).set_index(['Filme']).drop(filmesiguais).index
#essa lista vai localizar os filmes avaliados pelo usuario mais similar, classificar em relação à avaliação e eliminar os filmes que o user assistiu (filmesiguais)
print("Filmes indicados:")
for i in range(5):
    print(filmesindic[i],':',generos['Nome'][filmesindic[i]],'. Avaliação pelo usuário mais similar:', df.set_index(['Usuário']).loc[mostsimilaruser].set_index(['Filme']).loc[filmesindic[i]]['Avaliação'])


Usuário mais semelhante: 477
Filmes indicados:
815 : One Fine Day (1996) . Avaliação pelo usuário mais similar: 5
451 : Grease (1978) . Avaliação pelo usuário mais similar: 5
111 : Truth About Cats & Dogs, The (1996) . Avaliação pelo usuário mais similar: 5
49 : I.Q. (1994) . Avaliação pelo usuário mais similar: 5
722 : Nine Months (1995) . Avaliação pelo usuário mais similar: 5


3. Implementar um Sistema de Recomendações, usando a técnica de Filtros Colaborativos baseados na similaridade entre os itens (filmes);

In [71]:
nfilmes = len(generos) #quantidade de filmes
avgennorm = np.zeros((nuser,19)) #inicialização do vetor que possui dados normalizados do vetor avgen
for i in range(nuser):
    norm = max(avgen[i])
    avgennorm[i] = avgen[i]/norm
user = input("Insira um usuário:")
similaridadefilmes = np.zeros(nfilmes)
listageneros = generos.drop(['Nome','Data','','IMDB'],axis=1) #lista dos generos de cada filme
for i in range(nfilmes):
    similaridadefilmes[i] = cosinesim(avgennorm[int(user)-1],listageneros.loc[i+1])
filmesuser = list(df.set_index(['Usuário']).loc[int(user)]['Filme'])
filmesindic = pd.DataFrame(similaridadefilmes, columns = ['Similaridade']).sort_values(by = ['Similaridade'], ascending = False).drop(filmesuser).index
#classifica os 5 filmes mais similares
print('Filmes indicados:')
for i in range(5):
    print(filmesindic[i],':',generos['Nome'][filmesindic[i]])




Filmes indicados:
171 : Delicatessen (1991)
425 : Bob Roberts (1992)
559 : Interview with the Vampire (1994)
854 : Bad Taste (1987)
49 : I.Q. (1994)


In [11]:
def Pearson(A,B):
    sum = 0
    varA = 0
    varB = 0
    for i in range(len(A)):
        sum = sum + (A[i]-np.mean(A))*(B[i]-np.mean(B))
        modA = modA + (A[i]-np.mean(A))**2
        modB = modB + (B[i]-np.mean(B))**2
    cosine = sum/(math.sqrt(modA)*math.sqrt(modB))
    return cosine